Nog een keer hetzelfde programma alleen deze keer maak ik het met CustomTkinter. Dat is een Python UI library based on Tkinter die een nieuw, modernere widgets heeft.

In [1]:
# import sys
# !{sys.executable} -m pip install customtkinter

In [3]:
import tkinter
import customtkinter
import numpy as num
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

customtkinter.set_appearance_mode("Dark")  # Modes: system (default), light, dark
customtkinter.set_default_color_theme("green")  # Themes: blue (default), dark-blue, green

app = customtkinter.CTk()  # create CTk window like you do with the Tk window
app.geometry("400x240")

def button_function():
    print("button pressed")
    fig, ax = plt.subplots()
    col = ["JAAR_UITGAVE"]
    data = pd.read_csv('console.csv', usecols=col)
    xUnique = num.unique(data)
    count = data.value_counts()
    y = []
    print(xUnique)
    for i in xUnique:
        y.append(count.get(i))

    ax.bar(xUnique, y)
    ax.set_ylabel("Aantal")
    ax.set_xlabel("Jaar")
    ax.set_title("Aantal console releases per jaar")
        
        # globalefunctie defined in het laatste codescherm
    createWindow(fig)

def createWindow(figure):
        diaFrame = customtkinter.CTk()

        canvas = FigureCanvasTkAgg(figure, diaFrame)
        canvas.draw()

        canvas.get_tk_widget().pack(side=customtkinter.TOP, fill=customtkinter.BOTH, expand=1)

        diaFrame.mainloop()
 



# Use CTkButton instead of tkinter Button
button = customtkinter.CTkButton(master=app, text="CTkButton", command=button_function)
button.place(relx=0.5, rely=0.5, anchor=tkinter.CENTER)

app.mainloop()

button pressed
[2004 2011 2013 2014 2015 2016 2017]


CustomTkinter can adapt to the Windows 10/11 light or dark mode